In [ ]:
#| default_exp vitdet3d

# ViTDet 
- Vision Transformer for Detection 
- plain and non-hierarchical backbone. 
- Uses FPN 


### Some changes we made to the original ViTDetCode
- Simplified embeddings by removing class token and `get_absolute_positions`

In [ ]:
#| export 
import torch 
import torch.nn as nn
import math
import numpy as np
import collections
from typing import List, Union, Tuple, Optional, Dict

from transformers.models.vitdet.configuration_vitdet import VitDetConfig
from transformers.models.vitdet.modeling_vitdet import drop_path, VitDetDropPath, caffe2_msra_fill
from transformers.activations import ACT2FN
from transformers.modeling_outputs import BackboneOutput, BaseModelOutput
from transformers.modeling_utils import  PreTrainedModel
from transformers.utils.backbone_utils import BackboneMixin

In [ ]:
VitDetConfig()

VitDetConfig {
  "drop_path_rate": 0.0,
  "dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-06,
  "mlp_ratio": 4,
  "model_type": "vitdet",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "out_features": [
    "stage12"
  ],
  "out_indices": [
    12
  ],
  "patch_size": 16,
  "pretrain_image_size": 224,
  "qkv_bias": true,
  "residual_block_indices": [],
  "stage_names": [
    "stem",
    "stage1",
    "stage2",
    "stage3",
    "stage4",
    "stage5",
    "stage6",
    "stage7",
    "stage8",
    "stage9",
    "stage10",
    "stage11",
    "stage12"
  ],
  "transformers_version": "4.35.0",
  "use_absolute_position_embeddings": true,
  "use_relative_position_embeddings": false,
  "window_block_indices": [],
  "window_size": 0
}

## VitDetEmebddings. 

In [ ]:
image_size = (96, 192, 192)
patch_size = (4, 8, 8)
num_channels = 1
hidden_size = 96 
num_patches = (image_size[0]//patch_size[0]) * (image_size[1]//patch_size[1]) * (image_size[2]//patch_size[2])
num_positions = num_patches+1
num_positions

13825

In [ ]:
projection = nn.Conv3d(num_channels, hidden_size, kernel_size=patch_size, stride=patch_size)
projection

Conv3d(1, 96, kernel_size=(4, 8, 8), stride=(4, 8, 8))

In [ ]:
img = torch.randn((1, 1, )+image_size)
out = projection(img)
out.shape

torch.Size([1, 96, 24, 24, 24])

In [ ]:
embeddings = out.permute((0, 2, 3, 4, 1))
embeddings.shape

torch.Size([1, 24, 24, 24, 96])

In [ ]:
size = np.ceil(math.pow(num_patches, 1/3))
size*size*size == num_patches

True

In [ ]:
#| export 
# Copied from transformers.models.vitdet.modeling_vitdet.VitDetEmbeddings 
class ViTDet3dEmbeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
    
        image_size, patch_size = config.image_size, config.patch_size
        num_channels, hidden_size = config.num_channels, config.hidden_size

        image_size = image_size if isinstance(image_size, collections.abc.Iterable) else (image_size, image_size, image_size)
        patch_size = patch_size if isinstance(patch_size, collections.abc.Iterable) else (patch_size, patch_size, patch_size)
        num_patches = (image_size[2] // patch_size[2]) * (image_size[1] // patch_size[1]) * (image_size[0] // patch_size[0])
        self.image_size = image_size
        self.patch_size = patch_size
        self.num_channels = num_channels
        self.num_patches = num_patches

        if config.use_absolute_position_embeddings:
            # Initialize absolute positional embedding with pretrain image size.
            num_positions = num_patches + 1 # we are ignoring cls_token for now 
            self.position_embeddings = nn.Parameter(torch.zeros(1, num_patches, config.hidden_size))
        else:
            self.position_embeddings = None

        self.projection = nn.Conv3d(num_channels, hidden_size, kernel_size=patch_size, stride=patch_size)


    def forward(self, pixel_values: torch.Tensor) -> torch.Tensor:
        num_channels = pixel_values.shape[1]
        if num_channels != self.num_channels:
            raise ValueError(
                "Make sure that the channel dimension of the pixel values match with the one set in the configuration."
                f" Expected {self.num_channels} but got {num_channels}."
            )
        embeddings = self.projection(pixel_values)

        if self.position_embeddings is not None:
            # (batch_size, num_channels, depth, height, width) -> (batch_size, depth, height, width, num_channels)
            embeddings = embeddings.permute(0, 2, 3, 4, 1)
            # add position embeddings
            _, D, H, W , C = embeddings.shape
            embeddings = embeddings + self.position_embeddings.reshape((1, D, H, W, C))
            # (batch_size, depth, height, width, num_channels) -> (batch_size, num_channels, depth, height, width)
            embeddings = embeddings.permute(0, 4, 1, 2, 3)

        return embeddings

In [ ]:
config = VitDetConfig(image_size=(96, 192, 192), patch_size=(4, 8, 8), hidden_size=96, num_channels=1)
embed = ViTDet3dEmbeddings(config)
embed

ViTDet3dEmbeddings(
  (projection): Conv3d(1, 96, kernel_size=(4, 8, 8), stride=(4, 8, 8))
)

In [ ]:
embed.image_size

(96, 192, 192)

In [ ]:
img = torch.randn((1, 1, )+image_size)
print(img.shape)
embed_out = embed(img)
embed_out.shape

torch.Size([1, 1, 96, 192, 192])


torch.Size([1, 96, 24, 24, 24])

## Attention
- we have [qkv matrices]x n heads matrics 
- input is multiplied with qkv and we get qkv matrices 
- we split them and get qkv matrices 
- attention scores are generated

In [ ]:
hidden_states = embed_out.permute(0, 2, 3, 4, 1)
hidden_states.shape

torch.Size([1, 24, 24, 24, 96])

### Lets do windowing first 

In [ ]:
#| export 
# Copied from medct.swin.window_partition3d
def window_partition3d(input_feature, window_size: Union[int, List[int]]):
    """
    Partitions the given input into windows.
    """
    window_depth, window_height, window_width = window_size if isinstance(window_size, collections.abc.Iterable) else (window_size, window_size, window_size)
    batch_size, depth, height, width, num_channels = input_feature.shape
    input_feature = input_feature.view(
        batch_size, depth // window_depth, window_depth, height // window_height, window_height, width // window_width, window_width, num_channels
    )
    windows = input_feature.permute(0, 1, 3, 5, 2, 4, 6, 7).contiguous().view(-1, window_depth, window_height, window_width, num_channels)
    return windows

In [ ]:
window_size = (4, 4, 4)
windowed_hidden_states = window_partition3d(hidden_states, window_size)
windowed_hidden_states.shape

torch.Size([216, 4, 4, 4, 96])

> Apply attention now 

In [ ]:
dim = config.hidden_size
num_heads = config.num_attention_heads
head_dim = dim // num_heads
scale = head_dim**-0.5
print(dim, num_heads, dim, head_dim)
qkv = nn.Linear(dim, dim * 3, bias=config.qkv_bias)
qkv

96 12 96 8


Linear(in_features=96, out_features=288, bias=True)

In [ ]:
batch_size, depth, height, width, _ = windowed_hidden_states.shape
batch_size, depth, height, width

(216, 4, 4, 4)

In [ ]:
qkv_mat = qkv(hidden_states).reshape(batch_size, depth * height * width, 3, num_heads, -1).permute(2, 0, 3, 1, 4)
qkv_mat.shape

torch.Size([3, 216, 12, 64, 8])

In [ ]:
queries, keys, values = qkv_mat.reshape(3, batch_size * num_heads, depth* height * width, -1).unbind(0)
queries.shape, keys.shape, values.shape

(torch.Size([2592, 64, 8]),
 torch.Size([2592, 64, 8]),
 torch.Size([2592, 64, 8]))

In [ ]:
attention_scores = (queries * scale) @ keys.transpose(-2, -1)
attention_scores.shape #> this is a huge matrix and this is why we need window attention

torch.Size([2592, 64, 64])

### Now we have to add positional embeddings to the attention matrix 

In [ ]:
input_size = (4, 4, 4)

In [ ]:
rel_pos_d = nn.Parameter(torch.zeros(2 * input_size[0] - 1, head_dim))
rel_pos_h = nn.Parameter(torch.zeros(2 * input_size[1] - 1, head_dim))
rel_pos_w = nn.Parameter(torch.zeros(2 * input_size[2] - 1, head_dim))
rel_pos_d.shape, rel_pos_h.shape, rel_pos_w.shape

(torch.Size([7, 8]), torch.Size([7, 8]), torch.Size([7, 8]))

In [ ]:
q_size= k_size = (depth, height, width)
q_size, k_size, queries.shape

((4, 4, 4), (4, 4, 4), torch.Size([2592, 64, 8]))

In [ ]:
qh = q_size[1]
kh = k_size[1]
q_coords = torch.arange(qh)[:, None] * max(kh / qh, 1.0)
k_coords = torch.arange(kh)[None, :] * max(qh / kh, 1.0)
relative_coords = (q_coords - k_coords) + (kh - 1) * max(qh / kh, 1.0)
print(relative_coords.shape)
relative_coords.shape

torch.Size([4, 4])


torch.Size([4, 4])

In [ ]:
relative_coords

tensor([[3., 2., 1., 0.],
        [4., 3., 2., 1.],
        [5., 4., 3., 2.],
        [6., 5., 4., 3.]])

In [ ]:
#| export 
# copied from transformers.modelling.vitdet.get_rel_pos
def get_rel_pos(q_size, k_size, rel_pos):
    """
    Get relative positional embeddings according to the relative positions of query and key sizes.

    Args:
        q_size (`int`):
            Size of query q.
        k_size (`int`):
            Size of key k.
        rel_pos (`torch.Tensor`):
            Relative position embeddings (num_embeddings, num_channels).

    Returns:
        Extracted positional embeddings according to relative positions.
    """
    max_rel_dist = int(2 * max(q_size, k_size) - 1)
    # Interpolate rel pos if needed.
    if rel_pos.shape[0] != max_rel_dist:
        # Interpolate rel position embeddings.
        rel_pos_resized = torch.nn.functional.interpolate(
            rel_pos.reshape(1, rel_pos.shape[0], -1).permute(0, 2, 1),
            size=max_rel_dist,
            mode="linear",
        )
        rel_pos_resized = rel_pos_resized.reshape(-1, max_rel_dist).permute(1, 0)
    else:
        rel_pos_resized = rel_pos

    # Scale the coords with short length if shapes for q and k are different.
    q_coords = torch.arange(q_size)[:, None] * max(k_size / q_size, 1.0)
    k_coords = torch.arange(k_size)[None, :] * max(q_size / k_size, 1.0)
    relative_coords = (q_coords - k_coords) + (k_size - 1) * max(q_size / k_size, 1.0)

    return rel_pos_resized[relative_coords.long()]

In [ ]:
queries_depth, queries_height, queries_width = q_size
keys_depth, keys_height, keys_width = k_size
relative_depth = get_rel_pos(queries_depth, keys_depth, rel_pos_d)
relative_height = get_rel_pos(queries_height, keys_height, rel_pos_h)
relative_width = get_rel_pos(queries_width, keys_width, rel_pos_w)
relative_width.shape, relative_height.shape, relative_depth.shape

(torch.Size([4, 4, 8]), torch.Size([4, 4, 8]), torch.Size([4, 4, 8]))

In [ ]:
batch_size, _, dim = queries.shape
r_q = queries.reshape(batch_size, queries_depth, queries_height, queries_width, dim)
r_q.shape

torch.Size([2592, 4, 4, 4, 8])

#### torch.einsum

In [ ]:
x = torch.arange(20).view(5, 4)
print(x.shape)
x

torch.Size([5, 4])


tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]])

In [ ]:
y = torch.arange(20).view(4, 5)*1
print(y.shape)
y

torch.Size([4, 5])


tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])

In [ ]:
torch.einsum("ij, jk->ik", x, y)

tensor([[ 70,  76,  82,  88,  94],
        [190, 212, 234, 256, 278],
        [310, 348, 386, 424, 462],
        [430, 484, 538, 592, 646],
        [550, 620, 690, 760, 830]])

In [ ]:
torch.einsum("ij, jk->i", x, y)

tensor([ 410, 1170, 1930, 2690, 3450])

In [ ]:
x = torch.arange(100).view(5, 4, 5)
x.shape, x[0]

(torch.Size([5, 4, 5]),
 tensor([[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]]))

In [ ]:
y = torch.arange(125).view(5, 5, 5)*0 + 1
y.shape, y[0]

(torch.Size([5, 5, 5]),
 tensor([[1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1]]))

In [ ]:
torch.einsum("hwc,hkc->hwk", x, y)[0]

tensor([[10, 10, 10, 10, 10],
        [35, 35, 35, 35, 35],
        [60, 60, 60, 60, 60],
        [85, 85, 85, 85, 85]])

In [ ]:
relative_height = torch.einsum("bdhwc,hkc->bdhwk", r_q, relative_height)
relative_height.shape

torch.Size([2592, 4, 4, 4, 4])

In [ ]:
relative_depth = torch.einsum("bdhwc,dkc->bdhwk", r_q, relative_depth)
relative_width = torch.einsum("bdhwc,wkc->bdhwk", r_q, relative_width)
relative_width.shape, relative_depth.shape

(torch.Size([2592, 4, 4, 4, 4]), torch.Size([2592, 4, 4, 4, 4]))

In [ ]:
attn = attention_scores.view(batch_size, queries_depth, queries_height, queries_width, keys_depth, keys_height, keys_width)
attn.shape

torch.Size([2592, 4, 4, 4, 4, 4, 4])

In [ ]:
coords = (relative_depth[:, :, :, :, :, None, None] + relative_height[:, :, :, :, None, :, None] + relative_width[:, :, :, :, None, None, :])
coords.shape

torch.Size([2592, 4, 4, 4, 4, 4, 4])

In [ ]:
attn = (attn+coords).view(batch_size,  queries_depth * queries_height * queries_width, keys_depth *keys_height * keys_width)
attn.shape

torch.Size([2592, 64, 64])

In [ ]:
#| export 
#| copied from transformers.models.vitdet.add_decomposed_relative_positions 
def add_decomposed_relative_positions(attn, queries, rel_pos_d, rel_pos_h, rel_pos_w, q_size, k_size):
    """
    Calculate decomposed Relative Positional Embeddings as introduced in
    [MViT2](https://github.com/facebookresearch/mvit/blob/19786631e330df9f3622e5402b4a419a263a2c80/mvit/models/attention.py).

    Args:
        attn (`torch.Tensor`):
            Attention map.
        queries (`torch.Tensor`):
            Query q in the attention layer with shape (batch_size,queries_depth* queries_height * queries_width, num_channels).
        rel_pos_d (`torch.Tensor`):
            Relative position embeddings (Ld, num_channels) for depth axis.
        rel_pos_h (`torch.Tensor`):
            Relative position embeddings (Lh, num_channels) for height axis.
        rel_pos_w (`torch.Tensor`):
            Relative position embeddings (Lw, num_channels) for width axis.
        q_size (`Tuple[int]`):
            Spatial sequence size of query q with (queries_depth, queries_height, queries_width).
        k_size (`Tuple[int]`]):
            Spatial sequence size of key k with (keys_depth, keys_height, keys_width).

    Returns:
        attn (Tensor): attention map with added relative positional embeddings.
    """
    queries_depth, queries_height, queries_width = q_size
    keys_depth, keys_height, keys_width = k_size
    relative_depth = get_rel_pos(queries_depth, keys_depth, rel_pos_d)
    relative_height = get_rel_pos(queries_height, keys_height, rel_pos_h)
    relative_width = get_rel_pos(queries_width, keys_width, rel_pos_w)

    batch_size, _, dim = queries.shape
    r_q = queries.reshape(batch_size, queries_depth, queries_height, queries_width, dim)
    relative_depth = torch.einsum("bdhwc,dkc->bdhwk", r_q, relative_depth)
    relative_height = torch.einsum("bdhwc,hkc->bdhwk", r_q, relative_height)
    relative_width = torch.einsum("bdhwc,wkc->bdhwk", r_q, relative_width)

    attn = (
        attn.view(batch_size, queries_depth, queries_height, queries_width, keys_depth, keys_height, keys_width)
        + relative_depth[:, :, :, :, :, None, None]
        + relative_height[:, :, :, :, None, :, None]
        + relative_width[:, :, :, :, None, None, :]
    ).view(batch_size, queries_depth* queries_height * queries_width, keys_depth * keys_height * keys_width)
    return attn


## vit3dAttention

In [ ]:
#| export 
# copied from transformers.vitdet.vitdetattention
class VitDet3dAttention(nn.Module):
    """Multi-head Attention block with relative position embeddings."""

    def __init__(self, config, input_size=None):
        """
        Args:
            config (`VitDetConfig`):
                Model configuration.
            input_size (`Tuple[int]`, *optional*):
                Input resolution, only required in case relative position embeddings are added.
        """
        super().__init__()

        dim = config.hidden_size
        num_heads = config.num_attention_heads

        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim**-0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=config.qkv_bias)
        self.proj = nn.Linear(dim, dim)

        self.use_relative_position_embeddings = config.use_relative_position_embeddings
        if self.use_relative_position_embeddings:
            # initialize relative positional embeddings
            self.rel_pos_d = nn.Parameter(torch.zeros(2 * input_size[0] - 1, head_dim))
            self.rel_pos_h = nn.Parameter(torch.zeros(2 * input_size[1] - 1, head_dim))
            self.rel_pos_w = nn.Parameter(torch.zeros(2 * input_size[2] - 1, head_dim))

    def forward(self, hidden_state, output_attentions=False):
        batch_size, depth, height, width, _ = hidden_state.shape
        # qkv with shape (3, batch_size, num_heads, height * width, num_channels)
        qkv = self.qkv(hidden_state).reshape(batch_size, depth * height * width, 3, self.num_heads, -1).permute(2, 0, 3, 1, 4)
        # queries, keys and values have shape (batch_size * num_heads, height * width, num_channels)
        queries, keys, values = qkv.reshape(3, batch_size * self.num_heads, depth* height * width, -1).unbind(0)

        attention_scores = (queries * self.scale) @ keys.transpose(-2, -1)

        if self.use_relative_position_embeddings:
            attention_scores = add_decomposed_relative_positions(
                attention_scores, queries, self.rel_pos_d, self.rel_pos_h, self.rel_pos_w, (depth, height, width), (depth, height, width)
            )

        attention_probs = attention_scores.softmax(dim=-1)

        hidden_state = attention_probs @ values
        hidden_state = hidden_state.view(batch_size, self.num_heads, depth, height, width, -1)
        hidden_state = hidden_state.permute(0, 2, 3, 4, 1, 5)
        hidden_state = hidden_state.reshape(batch_size, depth, height, width, -1)
        hidden_state = self.proj(hidden_state)

        if output_attentions:
            attention_probs = attention_probs.reshape(
                batch_size, self.num_heads, attention_probs.shape[-2], attention_probs.shape[-1]
            )
            outputs = (hidden_state, attention_probs)
        else:
            outputs = (hidden_state,)

        return outputs

In [ ]:
config = VitDetConfig(image_size=(96, 192, 192), patch_size=(4, 8, 8), hidden_size=96, num_channels=1, use_relative_position_embeddings=True)
vdattn = VitDet3dAttention(config, input_size=window_size)
vdattn

VitDet3dAttention(
  (qkv): Linear(in_features=96, out_features=288, bias=True)
  (proj): Linear(in_features=96, out_features=96, bias=True)
)

In [ ]:
windowed_hidden_states.shape

torch.Size([216, 4, 4, 4, 96])

In [ ]:
out = vdattn(hidden_state=windowed_hidden_states)
out[0].shape

torch.Size([216, 4, 4, 4, 96])

In [ ]:
out = vdattn(hidden_state=windowed_hidden_states, output_attentions=True)
out[0].shape, out[1].shape

(torch.Size([216, 4, 4, 4, 96]), torch.Size([216, 12, 64, 64]))

## VitDetLayerNorm

In [ ]:
#| export 
# copied from transformers.vitdet.vitdetlayernorm
class VitDet3dLayerNorm(nn.Module):
    """
    A LayerNorm variant, popularized by Transformers, that performs point-wise mean and variance normalization over the
    channel dimension for inputs that have shape (batch_size, channels, height, width).
    https://github.com/facebookresearch/ConvNeXt/blob/d1fa8f6fef0a165b27399986cc2bdacc92777e40/models/convnext.py#L119
    """

    def __init__(self, normalized_shape, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.normalized_shape = (normalized_shape,)

    def forward(self, x):
        u = x.mean(1, keepdim=True)
        s = (x - u).pow(2).mean(1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.eps)
        x = self.weight[:, None, None, None] * x + self.bias[:, None, None, None]
        return x

## VitDetResNet

In [ ]:
#| export 
# copied from transformers.vitdet.vitdetresbottleneckblock
class VitDet3dResBottleneckBlock(nn.Module):
    """
    The standard bottleneck residual block without the last activation layer. It contains 3 conv layers with kernels
    1x1, 3x3, 1x1.
    """

    def __init__(self, config, in_channels, out_channels, bottleneck_channels):
        """
        Args:
            config (`VitDetConfig`):
                Model configuration.
            in_channels (`int`):
                Number of input channels.
            out_channels (`int`):
                Number of output channels.
            bottleneck_channels (`int`):
                Number of output channels for the 3x3 "bottleneck" conv layers.
        """
        super().__init__()
        self.conv1 = nn.Conv3d(in_channels, bottleneck_channels, 1, bias=False)
        self.norm1 = VitDet3dLayerNorm(bottleneck_channels)
        self.act1 = ACT2FN[config.hidden_act]

        self.conv2 = nn.Conv3d(bottleneck_channels, bottleneck_channels, 3, padding=1, bias=False)
        self.norm2 = VitDet3dLayerNorm(bottleneck_channels)
        self.act2 = ACT2FN[config.hidden_act]

        self.conv3 = nn.Conv3d(bottleneck_channels, out_channels, 1, bias=False)
        self.norm3 = VitDet3dLayerNorm(out_channels)

    def forward(self, x):
        out = x
        for layer in self.children():
            out = layer(out)

        out = x + out
        return out


## VitDetMLP

In [ ]:
#| export 
# copied from transformers.vitdet.vitdetmlp
class VitDet3dMlp(nn.Module):
    def __init__(self, config, in_features: int, hidden_features: int) -> None:
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = ACT2FN[config.hidden_act]
        self.fc2 = nn.Linear(hidden_features, in_features)
        self.drop = nn.Dropout(config.dropout_prob)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)

        return x

## Window Unpartition

In [ ]:
#| export 
# Copied from medct.swin.window_reverse3d
def window_reverse3d(windows, window_size, depth, height, width):
    """
    Merges windows to produce higher resolution features.
    """
    num_channels = windows.shape[-1]
    window_depth, window_height, window_width = window_size if isinstance(window_size, collections.abc.Iterable) else (window_size, window_size, window_size)
    windows = windows.view(-1, depth // window_depth, height // window_height, width // window_width, window_depth, window_height, window_width, num_channels)
    windows = windows.permute(0, 1, 3, 5, 2, 4, 6, 7).contiguous().view(-1, depth, height, width, num_channels)
    return windows

## VitDetLayer 

In [ ]:
#| export 
# copied from transformers.vitdet.vitdetlayer
class VitDet3dLayer(nn.Module):
    """This corresponds to the Block class in the original implementation."""

    def __init__(
        self, config, drop_path_rate: float = 0, window_size: Optional[Union[int, Tuple[int]]]= None, use_residual_block: bool = False
    ) -> None:
        super().__init__()

        dim = config.hidden_size
        input_size = (config.image_size[0] // config.patch_size[0], config.image_size[1] // config.patch_size[1], \
                      config.image_size[2] // config.patch_size[2])

        self.norm1 = nn.LayerNorm(dim, eps=config.layer_norm_eps)
        
        if window_size is None: window_size = (0, 0, 0)
        if isinstance(window_size, int): window_size = (window_size, window_size, window_size)
        
        self.attention = VitDet3dAttention(
            config, input_size=input_size if window_size[0] == 0 else window_size
        )

        self.drop_path = VitDetDropPath(drop_path_rate) if drop_path_rate > 0.0 else nn.Identity()
        self.norm2 = nn.LayerNorm(dim, eps=config.layer_norm_eps)
        self.mlp = VitDet3dMlp(config=config, in_features=dim, hidden_features=int(dim * config.mlp_ratio))

        self.window_size = window_size

        self.use_residual_block = use_residual_block
        if self.use_residual_block:
            # Use a residual block with bottleneck channel as dim // 2
            self.residual = VitDet3dResBottleneckBlock(
                config=config,
                in_channels=dim,
                out_channels=dim,
                bottleneck_channels=dim // 2,
            )

    def forward(
        self,
        hidden_states: torch.Tensor,
        head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Union[Tuple[torch.Tensor, torch.Tensor], Tuple[torch.Tensor]]:
        #TODO: this 
        hidden_states = hidden_states.permute(0, 2, 3, 4, 1)

        shortcut = hidden_states

        hidden_states = self.norm1(hidden_states)

        # Window partition
        if self.window_size[0] > 0:
            depth, height, width = hidden_states.shape[1], hidden_states.shape[2], hidden_states.shape[3]
            hidden_states = window_partition3d(hidden_states, self.window_size)

        self_attention_outputs = self.attention(
            hidden_states,
            output_attentions=output_attentions,
        )
        hidden_states = self_attention_outputs[0]
        outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        # Reverse window partition
        if self.window_size[0] > 0:
            hidden_states = window_reverse3d(hidden_states, self.window_size, depth, height, width)

        # first residual connection
        hidden_states = shortcut + self.drop_path(hidden_states)

        hidden_states = hidden_states + self.drop_path(self.mlp(self.norm2(hidden_states)))

        hidden_states = hidden_states.permute(0, 4, 1, 2, 3)

        if self.use_residual_block:
            hidden_states = self.residual(hidden_states)

        outputs = (hidden_states,) + outputs

        return outputs

In [ ]:
vdl = VitDet3dLayer(config, window_size=(4, 4, 4), use_residual_block=False)
vdl

VitDet3dLayer(
  (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
  (attention): VitDet3dAttention(
    (qkv): Linear(in_features=96, out_features=288, bias=True)
    (proj): Linear(in_features=96, out_features=96, bias=True)
  )
  (drop_path): Identity()
  (norm2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
  (mlp): VitDet3dMlp(
    (fc1): Linear(in_features=96, out_features=384, bias=True)
    (act): GELUActivation()
    (fc2): Linear(in_features=384, out_features=96, bias=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
)

In [ ]:
out = vdl(embed_out, output_attentions=True)
[i.shape for i in out], len(out)

([torch.Size([1, 96, 24, 24, 24]), torch.Size([216, 12, 64, 64])], 2)

## VitDet3dEncoder 

In [ ]:
config.window_block_indices, config.residual_block_indices

([], [])

In [ ]:
#| export 
# copied from transformers.vitdet.vitdetencoder
class VitDet3dEncoder(nn.Module):
    def __init__(self, config) -> None:
        super().__init__()
        self.config = config
        depth = config.num_hidden_layers

        # stochastic depth decay rule
        drop_path_rate = [x.item() for x in torch.linspace(0, config.drop_path_rate, depth)]

        layers = []
        for i in range(depth):
            layers.append(
                VitDet3dLayer(
                    config,
                    drop_path_rate=drop_path_rate[i],
                    window_size=config.window_size if i in config.window_block_indices else None,
                    use_residual_block=i in config.residual_block_indices,
                )
            )

        self.layer = nn.ModuleList(layers)
        self.gradient_checkpointing = False

    def forward(
        self,
        hidden_states: torch.Tensor,
        head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
        output_hidden_states: bool = False,
        return_dict: bool = True,
    ) -> Union[tuple, BaseModelOutput]:
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None

        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None

            if self.gradient_checkpointing and self.training:
                layer_outputs = self._gradient_checkpointing_func(
                    layer_module.__call__,
                    hidden_states,
                    layer_head_mask,
                    output_attentions,
                )
            else:
                layer_outputs = layer_module(hidden_states, layer_head_mask, output_attentions)

            hidden_states = layer_outputs[0]

            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(v for v in [hidden_states, all_hidden_states, all_self_attentions] if v is not None)
        return BaseModelOutput(
            last_hidden_state=hidden_states,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
        )

In [ ]:
config

VitDetConfig {
  "drop_path_rate": 0.0,
  "dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_size": 96,
  "image_size": [
    96,
    192,
    192
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-06,
  "mlp_ratio": 4,
  "model_type": "vitdet",
  "num_attention_heads": 12,
  "num_channels": 1,
  "num_hidden_layers": 12,
  "out_features": [
    "stage12"
  ],
  "out_indices": [
    12
  ],
  "patch_size": [
    4,
    8,
    8
  ],
  "pretrain_image_size": 224,
  "qkv_bias": true,
  "residual_block_indices": [],
  "stage_names": [
    "stem",
    "stage1",
    "stage2",
    "stage3",
    "stage4",
    "stage5",
    "stage6",
    "stage7",
    "stage8",
    "stage9",
    "stage10",
    "stage11",
    "stage12"
  ],
  "transformers_version": "4.35.0",
  "use_absolute_position_embeddings": true,
  "use_relative_position_embeddings": true,
  "window_block_indices": [],
  "window_size": 0
}

In [ ]:
config = VitDetConfig(image_size=(96, 192, 192), 
                      patch_size=(4, 8, 8), 
                      hidden_size=96,
                      num_channels=1,
                      use_relative_position_embeddings=True, 
                      window_block_indices=list(range(6)),
                      window_size =(4, 4, 4), 
                      out_indices = [6], 
                      num_hidden_layers= 6,
                      out_features = ["stage6"], 
                      stage_names = ["stem"]+[f"stage{i}" for i in range(1, 7)])
enc = VitDet3dEncoder(config)
enc

VitDet3dEncoder(
  (layer): ModuleList(
    (0-5): 6 x VitDet3dLayer(
      (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
      (attention): VitDet3dAttention(
        (qkv): Linear(in_features=96, out_features=288, bias=True)
        (proj): Linear(in_features=96, out_features=96, bias=True)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
      (mlp): VitDet3dMlp(
        (fc1): Linear(in_features=96, out_features=384, bias=True)
        (act): GELUActivation()
        (fc2): Linear(in_features=384, out_features=96, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
)

In [ ]:
enc.layer[0](embed_out)[0].shape

torch.Size([1, 96, 24, 24, 24])

In [ ]:
[i.window_size for i in enc.layer]

[(4, 4, 4), (4, 4, 4), (4, 4, 4), (4, 4, 4), (4, 4, 4), (4, 4, 4)]

In [ ]:
out = enc(embed_out, output_hidden_states=True, output_attentions=True)

In [ ]:
out.last_hidden_state.shape

torch.Size([1, 96, 24, 24, 24])

In [ ]:
[i.shape for i in out.attentions]

[torch.Size([216, 12, 64, 64]),
 torch.Size([216, 12, 64, 64]),
 torch.Size([216, 12, 64, 64]),
 torch.Size([216, 12, 64, 64]),
 torch.Size([216, 12, 64, 64]),
 torch.Size([216, 12, 64, 64])]

In [ ]:
[i.shape for i in out.hidden_states]

[torch.Size([1, 96, 24, 24, 24]),
 torch.Size([1, 96, 24, 24, 24]),
 torch.Size([1, 96, 24, 24, 24]),
 torch.Size([1, 96, 24, 24, 24]),
 torch.Size([1, 96, 24, 24, 24]),
 torch.Size([1, 96, 24, 24, 24]),
 torch.Size([1, 96, 24, 24, 24])]

## pretrained model 

In [ ]:
#| export 
# copied from transformers.vitdet.vitdetpretrainedmodel
class VitDet3dPreTrainedModel(PreTrainedModel):
    """
    An abstract class to handle weights initialization and a simple interface for downloading and loading pretrained
    models.
    """

    config_class = VitDetConfig
    base_model_prefix = "vitdet"
    main_input_name = "pixel_values"
    supports_gradient_checkpointing = True
    _no_split_modules = []

    def _init_weights(self, module: Union[nn.Linear, nn.Conv3d, nn.LayerNorm]) -> None:
        """Initialize the weights"""
        if isinstance(module, (nn.Linear, nn.Conv3d)):
            # Upcast the input in `fp32` and cast it back to desired `dtype` to avoid
            # `trunc_normal_cpu` not implemented in `half` issues
            module.weight.data = nn.init.trunc_normal_(
                module.weight.data.to(torch.float32), mean=0.0, std=self.config.initializer_range
            ).to(module.weight.dtype)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

        elif isinstance(module, ViTDet3dEmbeddings):
            module.position_embeddings.data = nn.init.trunc_normal_(
                module.position_embeddings.data.to(torch.float32),
                mean=0.0,
                std=self.config.initializer_range,
            ).to(module.position_embeddings.dtype)

        elif isinstance(module, VitDet3dAttention) and self.config.use_relative_position_embeddings:
            module.rel_pos_h.data = nn.init.trunc_normal_(
                module.rel_pos_h.data.to(torch.float32),
                mean=0.0,
                std=self.config.initializer_range,
            )
            module.rel_pos_w.data = nn.init.trunc_normal_(
                module.rel_pos_w.data.to(torch.float32),
                mean=0.0,
                std=self.config.initializer_range,
            )

        elif isinstance(module, VitDet3dResBottleneckBlock):
            for layer in [module.conv1, module.conv2, module.conv3]:
                caffe2_msra_fill(layer)
            for layer in [module.norm1, module.norm2]:
                layer.weight.data.fill_(1.0)
                layer.bias.data.zero_()
            # zero init last norm layer.
            module.norm3.weight.data.zero_()
            module.norm3.bias.data.zero_()

## VitDet3dModel

In [ ]:
#| export 
# copied from transformers.vitdet.vitdetmodel
class VitDet3dModel(VitDet3dPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.config = config

        self.embeddings = ViTDet3dEmbeddings(config)
        self.encoder = VitDet3dEncoder(config)

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self) -> ViTDet3dEmbeddings:
        return self.embeddings.projection

    def _prune_heads(self, heads_to_prune: Dict[int, List[int]]) -> None:
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer} See base
        class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)

    def forward(
        self,
        pixel_values: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, BaseModelOutput]:
        """
        Returns:

        Examples:

        ```python
        >>> from transformers import VitDetConfig, VitDetModel
        >>> import torch

        >>> config = VitDetConfig()
        >>> model = VitDetModel(config)

        >>> pixel_values = torch.randn(1, 3, 224, 224)

        >>> with torch.no_grad():
        ...     outputs = model(pixel_values)

        >>> last_hidden_states = outputs.last_hidden_state
        >>> list(last_hidden_states.shape)
        [1, 768, 14, 14]
        ```"""
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if pixel_values is None:
            raise ValueError("You have to specify pixel_values")

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(pixel_values)

        encoder_outputs = self.encoder(
            embedding_output,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]

        if not return_dict:
            return (sequence_output,) + encoder_outputs[1:]

        return BaseModelOutput(
            last_hidden_state=sequence_output,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
        )

In [ ]:
#| export 
# copied from transformers.vitdet.vitdetbackbone
class VitDet3dBackbone(VitDet3dPreTrainedModel, BackboneMixin):
    def __init__(self, config):
        super().__init__(config)
        super()._init_backbone(config)

        self.embeddings = ViTDet3dEmbeddings(config)
        self.encoder = VitDet3dEncoder(config)
        self.num_features = [config.hidden_size for _ in range(config.num_hidden_layers + 1)]

        # initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self) -> ViTDet3dEmbeddings:
        return self.embeddings.projection

    def forward(
        self,
        pixel_values: torch.Tensor,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> BackboneOutput:
        """
        Returns:

        Examples:

        ```python
        >>> from transformers import VitDetConfig, VitDetBackbone
        >>> import torch

        >>> config = VitDetConfig()
        >>> model = VitDetBackbone(config)

        >>> pixel_values = torch.randn(1, 3, 224, 224)

        >>> with torch.no_grad():
        ...     outputs = model(pixel_values)

        >>> feature_maps = outputs.feature_maps
        >>> list(feature_maps[-1].shape)
        [1, 768, 14, 14]
        ```"""
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions

        embedding_output = self.embeddings(pixel_values)

        outputs = self.encoder(
            embedding_output,
            output_hidden_states=True,
            output_attentions=output_attentions,
            return_dict=return_dict,
        )

        hidden_states = outputs.hidden_states if return_dict else outputs[1]

        feature_maps = ()
        for stage, hidden_state in zip(self.stage_names, hidden_states):
            if stage in self.out_features:
                feature_maps += (hidden_state,)

        if not return_dict:
            if output_hidden_states:
                output = (feature_maps,) + outputs[1:]
            else:
                output = (feature_maps,) + outputs[2:]
            return output

        return BackboneOutput(
            feature_maps=feature_maps,
            hidden_states=outputs.hidden_states if output_hidden_states else None,
            attentions=outputs.attentions,
        )

In [ ]:
enc = VitDet3dBackbone(config)
enc

VitDet3dBackbone(
  (embeddings): ViTDet3dEmbeddings(
    (projection): Conv3d(1, 96, kernel_size=(4, 8, 8), stride=(4, 8, 8))
  )
  (encoder): VitDet3dEncoder(
    (layer): ModuleList(
      (0-5): 6 x VitDet3dLayer(
        (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
        (attention): VitDet3dAttention(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
        (mlp): VitDet3dMlp(
          (fc1): Linear(in_features=96, out_features=384, bias=True)
          (act): GELUActivation()
          (fc2): Linear(in_features=384, out_features=96, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )
  )
)

In [ ]:
img = torch.randn((1, 1, 96, 192, 192))
print(img.shape)
out = enc(img)

torch.Size([1, 1, 96, 192, 192])


In [ ]:
[i.shape for i in out.feature_maps]

[torch.Size([1, 96, 24, 24, 24])]

In [ ]:
out.hidden_states

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()